# 문서 로더(document Loader)
### Document 객체
: LangChain에서 Document 객체는 모든 데이터의 기본 단위이며, 
  Loader로 불러온 텍스트를 모델이 이해할 수 있는 표준 구조로 정리한 클래스

In [19]:
import os
from dotenv import load_dotenv
load_dotenv("/home/ubuntu/project/.env")

True

In [11]:
from langchain_core.documents import Document
# Document는 LangChain 라이브러리의 일부로, 다음과 같이 import 해야 합니다 👇

In [12]:
docs = \
[
  Document(
    page_content="1장. LangChain 개요...",
    metadata={"source": "data/guide.pdf", "page": 1}
  ),
  Document(
    page_content="2장. Document Loader의 종류...",
    metadata={"source": "data/guide.pdf", "page": 2}
  ),
]
print(docs[0])
print(docs[1])

page_content='1장. LangChain 개요...' metadata={'source': 'data/guide.pdf', 'page': 1}
page_content='2장. Document Loader의 종류...' metadata={'source': 'data/guide.pdf', 'page': 2}


In [15]:
metadata={"source": "lecture_note", "page": 1}

In [16]:
print('page_content:',docs[0].page_content)
print('metadata:',docs[0].metadata)

page_content: 1장. LangChain 개요...
metadata: {'source': 'data/guide.pdf', 'page': 1}


In [2]:
%%writefile sample.txt
이 문서는 Langchain TextLoader 예시입니다.
여러 줄의 텍스트를 포함합니다.

Writing sample.txt


In [24]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


In [25]:
#1) textload - from langchain
loader = TextLoader("sample.txt", encoding="utf-8")
docs = loader.load()

#2) 청크 단위로 나누기
splitter = RecursiveCharacterTextSplitter(chunk_size = 50, chunk_overlap = 10)
split_docs = splitter.split_documents(docs)

In [27]:
pip install -U faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.4 MB/s  0:00:00 eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
#3) 임베딩 생성 및 벡터 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(split_docs,embeddings)

In [30]:
print("총", len(split_docs), "개의 청크가 생성되었습니다")

총 2 개의 청크가 생성되었습니다


###  [2] DirectoryLoader
: DirectoryLoader는 폴더(디렉터리) 안의 여러 파일을 자동으로 탐색하여 <br>
각 파일을 개별 Document 객체로 읽어들이는 문서 로더(Document Loader) 이다. <br>
이 클래스는 실제 프로젝트에서 대규모 문서 일괄 처리 시 거의 항상 사용되는 핵심 도구이다.

In [31]:
! mkdir data

In [32]:
# 1. data 폴더 생성 (이미 존재하면 무시)
os.makedirs("data", exist_ok=True)

# 2. 샘플 텍스트 데이터
samples = {
    "sample1.txt": """LangChain은 LLM(대규모 언어모델)을 활용한 파이프라인 구축을 위한 오픈소스 프레임워크입니다.
Document Loader를 통해 다양한 외부 데이터를 로드하고,
Text Splitter로 문서를 나눈 후, Embedding 및 VectorStore를 이용해 RAG 시스템을 구성할 수 있습니다.""",

    "sample2.txt": """FAISS는 Facebook AI Research에서 개발한 벡터 검색 라이브러리입니다.
LangChain에서는 문서 임베딩을 벡터로 변환한 후,
FAISS를 이용해 빠른 유사도 검색을 수행합니다.""",

    "sample3.txt": """pgvector는 PostgreSQL 데이터베이스에서 벡터 데이터를 저장하고 검색할 수 있게 해주는 확장 모듈입니다.
LangChain은 pgvector를 통해 RAG 시스템을 SQL 기반 환경에서도 구현할 수 있습니다."""
}

# 3. 파일 생성 및 저장
for filename, content in samples.items():
    path = os.path.join("data", filename)
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    print(f" {filename} 파일 생성 완료")

print("\n📂 'data' 폴더에 3개의 샘플 텍스트 파일이 준비되었습니다.")

 sample1.txt 파일 생성 완료
 sample2.txt 파일 생성 완료
 sample3.txt 파일 생성 완료

📂 'data' 폴더에 3개의 샘플 텍스트 파일이 준비되었습니다.
